In [13]:
from hamiltonian import Hamiltonian, HamiltonianH2, HamiltonianW

from sparse import ground, energy
from variance import variance_local, variance

from var_optimise import optimal_beta

In [2]:
lih = Hamiltonian('LiH', 1.5)
beh2 = Hamiltonian('BeH2', 1.3)

h2_jw = HamiltonianH2('jw')
h2_parity = HamiltonianH2('parity')
h2_bk = HamiltonianH2('bk')

water_jw = HamiltonianW('jw')
water_parity = HamiltonianW('parity')
water_bk = HamiltonianW('bk')

In [ ]:
%time energy, state = ground(lih.pauli_rep)
%time energy, state = ground(beh2.pauli_rep)
%time energy, state = ground(h2_jw.pauli_rep)
%time energy, state = ground(water_jw.pauli_rep)

In [3]:
pr = beh2.pauli_rep
energy1, state = ground(pr)
print("energy1: ", energy1)
energy2 = energy(pr, state)
print("energy2: ", energy2)

energy1:  -3.989368880501249
energy2:  -3.9893688805012504


In [5]:
%time variance_local(pr, energy1, state, pr.local_dists_uniform())

CPU times: user 10.6 s, sys: 28.6 ms, total: 10.6 s
Wall time: 10.6 s


18.1798507944044

In [14]:
print(variance(pr, energy1, state, 'ell_1'))
print(variance_local(pr, energy1, state, pr.local_dists_pnorm(1)))
print(variance_local(pr, energy1, state, pr.local_dists_pnorm(2)))
print(variance_local(pr, energy1, state, pr.local_dists_pnorm('infinity')))

42.28949301890355
88.57935992546831
80.69169432668951
79.62377675669399


# Variance optimisation

If we use a uniform/biased pauli/magic bases algorithm to estimate the energy, and we assume that the state is a product-state (or close to a produce-state) then we end up wanting to minimise:
$$
    \Var[\nu]
    =
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    +
    \sum_{\Qarrow,\Rarrow \in \{I,Z\}^{\otimes n}} \alpha_\Qarrow \alpha_\Rarrow \prod_{i} m_i(1-\delta_{Q_i R_i})
    - \tr(H_0\rho)^2
$$
(Above is the chemistry HF situation, where the product-state is in the $Z$-basis, but this statement wouldn't depend on using $Z$-basis uniformly.)

We end up wanting to minimise:
$$
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    \qquad
    \textrm{subject to}
    \qquad
    \beta_{i,X}+\beta_{i,Y}+\beta_{i,Z}=1 \,\forall i,
    \qquad
    \beta_{i,P}\ge 0
$$

In [16]:
β_optimal = optimal_beta(pr)
print(variance_local(pr, energy1, state, β_optimal))

13.845088491932884


In [21]:
def var_dic_and_opt_beta(pauli_rep):
    pr = pauli_rep
    dic = {}
    
    energy1, state = ground(pr)
    print("energy :", energy1)

    # ell_1
    var = variance(pr, energy1, state, 'ell_1')
    print("ell 1: ", var)
    dic['ell_1'] = var
    
    # uniform
    var = variance_local(pr, energy1, state, pr.local_dists_uniform())
    print("uniform: ", var)
    dic['uniform'] = var
    
    # optimal
    β_optimal = optimal_beta(pr)
    var = variance_local(pr, energy1, state, β_optimal)
    print("optimal: ", var)
    dic['optimal'] = var
    
    return dic, β_optimal

In [27]:
pr = lih.pauli_rep
lih_dic, lih_beta = var_dic_and_opt_beta(pr)

energy : -1.1014068450598966
ell 1:  8.798891105500473
uniform:  2.230730161038924
optimal:  1.576299391494918


In [30]:
pr = beh2.pauli_rep
beh2_dic, beh2_beta = var_dic_and_opt_beta(pr)

energy : -3.989368880501248
ell 1:  42.28949301890355
uniform:  18.179850794404405
optimal:  13.845088491932845


In [32]:
pr = h2_jw.pauli_rep
h2_jw_dic, h2_jw_beta = var_dic_and_opt_beta(pr)

energy : -1.8608605555207645
ell 1:  119.67906001905907
uniform:  51.399820213876076
optimal:  17.74207193681142


In [ ]:
pr = h2_parity.pauli_rep
h2_parity_dic, h2_parity_beta = var_dic_and_opt_beta(pr)

In [38]:
pr = h2_bk.pauli_rep
h2_bk_dic, h2_bk_beta = var_dic_and_opt_beta(pr)

energy : -1.8608605555207596
ell 1:  119.67906001905943
uniform:  168.86600324405455
optimal:  19.516932420931536


In [41]:
pr = water_jw.pauli_rep

In [42]:
β = optimal_beta(pr)

/opt/anaconda3/envs/qiskit18/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [43]:
β

{0: [0.43257858556565915, 0.43257858722135467, 0.1348428272129862],
 1: [0.24098007537433425, 0.2409800708577077, 0.518039853767958],
 2: [0.13870541903721406, 0.13870542588015417, 0.7225891550826317],
 3: [0.13957872152763764, 0.13957871134125624, 0.7208425671311062],
 4: [0.16984472160573807, 0.1698447243114196, 0.6603105540828423],
 5: [0.2675208449775755, 0.2675208434379076, 0.464958311584517],
 6: [0.11647067744295292, 0.11647067819430176, 0.7670586443627454],
 7: [0.43257857708372555, 0.43257860312684154, 0.13484281978943294],
 8: [0.24098005034367995, 0.24098008250406633, 0.5180398671522537],
 9: [0.13870542248842968, 0.13870542724539398, 0.7225891502661764],
 10: [0.13957872657623138, 0.13957872195005588, 0.7208425514737127],
 11: [0.16984475333796817, 0.16984473040786618, 0.6603105162541657],
 12: [0.2675208758292949, 0.26752084565650264, 0.46495827851420257],
 13: [0.11647066971784335, 0.1164706702788042, 0.7670586600033525]}

In [44]:
pr.local_dists_pnorm(1)

{0: [0.2670624598738536, 0.2670624598738536, 0.4658750802522929],
 1: [0.2183174172606949, 0.2183174172606949, 0.5633651654786103],
 2: [0.03083301822077977, 0.03083301822077977, 0.9383339635584405],
 3: [0.14333043314617216, 0.14333043314617216, 0.7133391337076557],
 4: [0.16199346438598008, 0.16199346438598008, 0.6760130712280399],
 5: [0.22801250514720012, 0.22801250514720012, 0.5439749897055998],
 6: [0.04542591659480066, 0.04542591659480066, 0.9091481668103987],
 7: [0.2670624598738535, 0.2670624598738535, 0.46587508025229296],
 8: [0.21831741726069484, 0.21831741726069484, 0.5633651654786103],
 9: [0.03083301822077977, 0.03083301822077977, 0.9383339635584405],
 10: [0.1433304331461722, 0.1433304331461722, 0.7133391337076557],
 11: [0.16199346438598014, 0.16199346438598014, 0.6760130712280398],
 12: [0.22801250514720015, 0.22801250514720015, 0.5439749897055998],
 13: [0.04542591659480068, 0.04542591659480068, 0.9091481668103987]}

In [45]:
energy1, state = ground(pr)

In [46]:
var = variance_local(pr, energy1, state, pr.local_dists_uniform())
print("uniform: ", var)

uniform:  2839.039468218853


In [47]:
variance(pr, energy1, state, 'ell_1')

4363.497773126066

In [48]:
var2 = variance_local(pr, energy1, state, β)
print("optimal: ", var2)

optimal:  257.5450993727012
